In [15]:
from physcraper import opentree_helpers
from opentree import OT
import dendropy
import json
import sys


In [9]:
# Input alignemnt from treebase stored at https://github.com/McTavishLab/physcraper_example/blob/master/treebase_alns/pg_55tree5864_ndhf.aln
# Data files created by running:
# physcraper_run.py -s pg_55 -t tree5864 -a treebase_alns/pg_55tree5864_ndhf.aln -as "nexus" -db ~/ncbi/localblastdb/ -o pg_55


In [10]:
updated_tre_path = "pg_55/outputs_pg_55tree5864_ndhf/physcraper_pg_55tree5864_ndhf.tre"
otu_json_path = "pg_55/run_pg_55tree5864_ndhf/otu_info_pg_55tree5864_ndhf.json"

In [11]:
updated_tre = dendropy.Tree.get(path=updated_tre_path, schema = "newick")
otu_dict = json.load(open(otu_json_path, "r"))

In [23]:
#Choose outgroups and root
outgroup = []

In [28]:
tree_conf = opentree_helpers.conflict_tree(updated_tre, otu_dict)
print(tree_conf)

((_nd4_ott536207:0.003911694828330017,(((((_nd10_ott5551:0.0004593978522760431,_nd11_ott5551:0.006807994474845579)_nd9_:0.0024316068078051715,((_nd14_ott269819:0.00096967766451874,_nd15_ott269817:0.000480490847689628)_nd13_:0.0029006913045804567,(_nd17_ott793340:1.00000050002909e-06,_nd18_ott269815:0.0024882395706607476)_nd16_:0.0029331574751482532)_nd12_:1.00000050002909e-06)_nd8_:0.0004809847851856634,((_nd21_ott536243:0.003381028280635386,_nd22_ott108610:0.005174476457159359)_nd20_:0.0011003046554364077,(_nd24_ott638567:0.0009641915804151977,(_nd26_ott638569:0.0019341992464948308,_nd27_ott638567:0.0009547274669291637)_nd25_:1.00000050002909e-06)_nd23_:0.0033908362263461774)_nd19_:0.007332891570371064,(_nd29_ott595902:0.0019807536523385265,(_nd31_ott861917:0.0029018308796110856,(_nd33_ott578275:0.013118919150435926,(_nd35_ott312752:0.003953881957702714,_nd36_ott84784:0.005904945692878712)_nd34_:1.00000050002909e-06)_nd32_:0.0024140122068815367,(_nd38_ott827914:0.003414405886291387,((

In [13]:
resp_updated = OT.conflict_str(tree_updated.as_string(schema="newick"), 'ott')

In [27]:
conflict = resp_updated.response_dict
conflict

{'nd10': {'status': 'terminal',
  'witness': 'ott5551',
  'witness_name': 'Brachychiton acerifolius'},
 'nd12': {'status': 'resolved_by',
  'witness': 'ott996482',
  'witness_name': 'Sterculioideae'},
 'nd13': {'status': 'supported_by',
  'witness': 'ott269823',
  'witness_name': 'Acropogon (genus in kingdom Archaeplastida)'},
 'nd14': {'status': 'terminal',
  'witness': 'ott269819',
  'witness_name': 'Acropogon bullatus'},
 'nd15': {'status': 'terminal',
  'witness': 'ott269817',
  'witness_name': 'Acropogon dzumacensis'},
 'nd16': {'status': 'resolved_by',
  'witness': 'ott996482',
  'witness_name': 'Sterculioideae'},
 'nd17': {'status': 'terminal',
  'witness': 'ott793340',
  'witness_name': 'Franciscodendron laurifolium'},
 'nd18': {'status': 'terminal',
  'witness': 'ott269815',
  'witness_name': 'Argyrodendron peralatum'},
 'nd19': {'status': 'supported_by',
  'witness': 'ott526990',
  'witness_name': 'Heritiera'},
 'nd20': {'status': 'resolved_by',
  'witness': 'ott526990',
  'w

In [16]:


updated_conf_taxa = set()
for node in conflict:
    if conflict[node]['status'] == 'conflicts_with':
        witness = conflict[node]['witness_name']
        updated_conf_taxa.add(witness)

sys.stdout.write("Updated tree conflicts with {} taxa in the OpenTree taxonomy:\n".format(len(updated_conf_taxa)))
for tax in updated_conf_taxa:
    sys.stdout.write("{}\n".format(tax))


Updated tree conflicts with 4 taxa in the OpenTree taxonomy:
Bombacoideae
Sterculioideae
Scaphium (genus in kingdom Archaeplastida)
Firmiana + Hildegardia (genus in kingdom Archaeplastida)


In [24]:
## Relabel tree with conflict information
for node in tree_conf:
    if node.taxon:
        node_id = node.taxon.label.split('_')[1]
        conf_node = conflict.get(node_id, False)
        if conf_node:
            new_label = "{} {}".format(conf_node['status'], conf_node['witness_name'])
            node.taxon.label = new_label
    else:
        node_id = node.label.split('_')[1]
        conf_node = conflict.get(node_id, False)
        if conf_node:
            new_label = "{} {}".format(conf_node['status'], conf_node['witness_name'])
            node.label = new_label


In [25]:
tree_conf.as_ascii_plot()

'              /------------------------------------------------------------------------------------------------------------------------------------------------------------- terminal Octolobus spectabilis       \n              |                                                                                                                                                                                                   \n              |                                                                                                                                              /-------------- terminal Brachychiton acerifolius    \n              |                                                                                                                  /---------------------------+                                                    \n              |                                                                                                                  |                     